In [15]:
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

import pandas as pd
from matplotlib import pyplot as plt
import random
import time
import models
import gdown

from scipy.stats import nbinom,norm,truncnorm
from scipy.integrate import trapezoid as trapz

In [16]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTltw5xQVB_sSBCLiA5nzbiDc1srkInw5TDBWcYy50A-Yhr7zKMTgJUy0aoE1q0uCo5WUJSJSVhR_SY/pub?gid=1190540286&single=true&output=csv"
df = pd.read_csv(url)

In [17]:
#file_path = 'results.ods'

# Read the Excel file
#df = pd.read_excel(file_path, engine="odf")

# Print the DataFrame
print(df)

winners=np.array(df['Winner\'s name'])
losers=np.array(df['Loser\'s name'])
loser_scores=np.array(df['Loser\'s score'])
names = pd.concat([df['Winner\'s name'], df['Loser\'s name']]).unique()
ids = list(range(len(names)))
dict_to_id = dict(zip(names,ids))
dict_to_name=dict(zip(ids,names))
winner_ids = np.array([dict_to_id[i] for i in winners])
loser_ids = np.array([dict_to_id[i] for i in losers])

     Chronological info Winner's name Loser's name  Loser's score
0   09/08/2023 15.08.50          Fedo      Alfredo             10
1   09/08/2023 15.09.11       Alfredo         Fedo              5
2   09/08/2023 15.09.27          Fedo      Alfredo              8
3   09/08/2023 17.08.36        Younes       Andrea              3
4   10/08/2023 13.44.32       Alfredo        Idris              9
5   10/08/2023 13.44.48       Alfredo        Idris              8
6   10/08/2023 13.45.05       Alfredo        Idris              7
7   10/08/2023 15.13.32         Idris       Satvik              7
8                   NaN          Fedo        Mauro              1
9                   NaN          Fedo        Idris              3
10                  NaN         Mauro        Idris              6
11                  NaN          Fedo        Idris              5
12                  NaN         Mauro        Idris              8
13                  NaN          Fedo        Mauro              5
14        

## Gibbs sampling

In [18]:
#just in case you run this model after the previous one
if isinstance(winner_ids, torch.Tensor):
    winner_ids=winner_ids.numpy()
    loser_ids=loser_ids.numpy()
    loser_scores=loser_scores.numpy()

In [19]:
g=models.gibbs_model(winner_ids, loser_ids, loser_scores,names=names,prior_std=0.3)

In [20]:
g.posterior_sampling(n_iterations=500, warmup=100)

Progress: [################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################### ] 99%

In [21]:
g.print_table()

Player 1: Fedo
Posterior mean of strength: 0.9385736380665993
Posterior std dev of strength: 0.04346656020732055

Player 2: Alfredo
Posterior mean of strength: 0.7072876866111646
Posterior std dev of strength: 0.11779589328000199

Player 3: Younes
Posterior mean of strength: 0.4784300019083391
Posterior std dev of strength: 0.12193270797166586

Player 4: Idris
Posterior mean of strength: 0.521056931942621
Posterior std dev of strength: 0.06349582529993336

Player 5: Mauro
Posterior mean of strength: 0.7964914924123819
Posterior std dev of strength: 0.1027118546673527

Player 6: Frago
Posterior mean of strength: 0.7477736594187279
Posterior std dev of strength: 0.1087548005272227

Player 7: Nour
Posterior mean of strength: 0.46857043683137406
Posterior std dev of strength: 0.08920426375481928

Player 8: Andrea
Posterior mean of strength: 0.3961830763756729
Posterior std dev of strength: 0.0951837858257769

Player 9: Satvik
Posterior mean of strength: 0.364181597956606
Posterior std dev 

In [ ]:
def unnorm_product_function(l1,lambdas_wins, lambdas_losses, scores, std=0.2, epsilon=1e-12):
    if l1<0 or l1>1:
        return 0
    else:
        #p=np.concatenate(((l1+epsilon)/(lambdas_wins+l1+epsilon),(lambdas_losses+epsilon)/(lambdas_losses+l1+epsilon)))
        p=np.concatenate(((l1+epsilon)/(lambdas_losses+l1+epsilon),(lambdas_wins+epsilon)/(lambdas_wins+l1+epsilon)))
        func=np.prod(nbinom.pmf(11,scores,p))*norm.pdf(l1,0.5,std)
        return func


def slice_sampler(x_init=0.5,custom_function=unnorm_product_function, n_samples=10):
    x = x_init
    samples = np.zeros(n_samples)
    for i in range(n_samples):
        # Draw a vertical line
        y = np.random.uniform(0, custom_function(x))
        # Create a horizontal “slice” (i.e., an interval)
        x_left = x - 0.1
        while y < custom_function(x_left):
            x_left -= 0.1
        x_right = x + 0.1
        while y < custom_function(x_right):
            x_right += 0.1
        # Draw new sample
        if x_left<0:
            x_left=0.01
        if x_right>1:
            x_right=0.99
        #print(x_left,x_right)
        while True:
            x_new = np.random.uniform(x_left, x_right)
            if y < custom_function(x_new):
                break
            elif x_new > x:
                x_right = x_new
            elif x_new < x:
                x_left = x_new
            else:
                raise Exception("Slice sampler shrank to zero!")
        x = x_new
        samples[i] = x
    return samples[-1]

def init_strengths(n,std):
    mean = 0.5
    # replace with your value
    lower, upper = 0, 1
    # Because truncnorm takes its bounds in the standard Gaussian space, 
    # you should convert your bounds
    lower_std, upper_std = (lower - mean) / std, (upper - mean) / std
    # Create the truncated Gaussian distribution
    truncated_gaussian = truncnorm(lower_std, upper_std, loc=mean, scale=std)
    # Sample from the distribution
    sample = truncated_gaussian.rvs(n)
    return sample

def simulate_match(p):
    count_heads = np.random.binomial(11, p)
    count_tails = 11-count_heads
    while count_heads < 11 and count_tails < 11:
        # draw a sample from a Bernoulli distribution
        sample = np.random.binomial(1, p)
        if sample == 1:
            count_heads += 1
        else:
            count_tails += 1
    if count_heads ==11:
        return 1
    else:
        return 0

class gibbs_model:
    def __init__(self, winner_ids, loser_ids, loser_scores, prior_std=0.2,names=None):
        #find the number of players
        self.n_players=len(np.unique(np.concatenate((winner_ids,loser_ids))))
        #initialize the strengths; you can also put a wider prior
        self.strengths=init_strengths(self.n_players,prior_std)
        self.winner_ids=winner_ids
        self.loser_ids=loser_ids
        self.loser_scores=loser_scores
        self.prior_std=prior_std
        self.names=names
        self.posterior_samples=None
        
    def posterior_sampling(self,n_iterations=100, warmup=20):
        strengths_time=[]
        ids = list(range(self.n_players))
        thermalized=False
        for i in range(n_iterations):
            print("\r" + "Progress: [" + "#" * i + " " * (n_iterations - i) + f"] {int(100 * i / n_iterations)}%", end="")
            if thermalized==False:
                if i==warmup:
                    thermalized=True
            random.shuffle(ids)
            for current_id in ids:
                #print(current_id)
                l1=self.strengths[current_id]
                #print(l1)
                #print(np.where(self.winner_ids == current_id))
                lambdas_wins=self.strengths[self.loser_ids[np.where(self.winner_ids == current_id)]]
                scores=self.loser_scores[np.where(self.winner_ids == current_id)]
                lambdas_losses=self.strengths[self.winner_ids[np.where(self.loser_ids == current_id)]]
                scores=np.concatenate((scores,self.loser_scores[np.where(self.loser_ids == current_id)]))
                custom_function = lambda l1: unnorm_product_function(l1, lambdas_wins, lambdas_losses, scores, self.prior_std, epsilon)
                self.strengths[current_id] = slice_sampler(x_init=self.strengths[current_id], custom_function=custom_function)
                #self.strengths[current_id]=slice_sampler()
            if thermalized:
                strengths_time.append(self.strengths.copy())
        self.posterior_samples=np.array(strengths_time)
        
    def predict_match(self, player_1_id, player_2_id, n_matches=1000, explicit=False):
        player_1_wins=0
        l1=np.random.choice(self.posterior_samples[:,player_1_id], size=n_matches, replace=True, p=None)
        l2=np.random.choice(self.posterior_samples[:,player_2_id], size=n_matches, replace=True, p=None)
        for match in range(n_matches):
            player_1_wins+=simulate_match(l1[match]/(l1[match]+l2[match]))
        ratio=player_1_wins/n_matches
        if explicit:
            print(f'Player 1 has {ratio*100}% probability of winning')
        return ratio
    
    def print_table(self):
        # Print the posterior mean and standard deviation of player strengths
        mean=np.mean(self.posterior_samples,axis=0)
        std=np.std(self.posterior_samples,axis=0)
        for i in range(self.n_players):
            if self.names is not None:
                player = f'Player {i+1}: {self.names[i]}'
            else:
                player = f'Player {i+1}'
            print(player)
            print("Posterior mean of strength:", mean[i])
            print("Posterior std dev of strength:", std[i])
            print()
            
        for i in range(self.n_players):
            for j in range(self.n_players)[i+1:]:
                if self.names is not None:
                    print(f'{self.names[i]} has {int(self.predict_match(i,j)*100)}% of probability of winning against {self.names[j]}')
                else:
                    print(f'Player {i+1} has {int(self.predict_match(i,j)*100)}% of probability of winning against Player {j+1}')